In [ ]:
# https://adventofcode.com/2018/day/15

using DataStructures

mutable struct Unit
    pos::CartesianIndex{2}
    race::Int
    hp::Int
    ap::Int
end

data = hcat(open("15.txt", "r") do file
    readlines(file) .|> line -> [convert(Int, c) for c in line]
end...)'

const WALL, FLOOR, ELF, GOBLIN = convert.(Int, collect("#.EG"))


In [ ]:
# enemy check
isenemy(grid, a, b) = Dict(ELF=>GOBLIN, GOBLIN=>ELF)[grid[a]] == grid[b]

# manhattan grid distance
distance(a, b) = abs(a[1] - b[1]) + abs(a[2] - b[2])

# uniform cost search algorithm
function search(grid, start)
    frontier = [start]
    explored = Dict()
    nearest = []
    
    while !isempty(frontier)
        # if we find at least one enemy neighbor, we are done
        nearest = frontier[[isenemy(grid, start, n) for n in frontier]]
        if !isempty(nearest) break end
        
        # find the valid immediate neighbors in reading order
        # don't traverse walls or units of our same race,
        # and ignore any nodes we have already visited
        node = popfirst!(frontier)
        y, x = Tuple(node)
        neighbors = [n for n in CartesianIndex.([(y - 1, x), 
                                                 (y, x - 1), 
                                                 (y, x + 1), 
                                                 (y + 1, x)])
                     if grid[n] ∉ [WALL, grid[start]] && !haskey(explored, n)]

        # add the current node to the path dict
        # and add the neighbors to the frontier
        for neighbor in neighbors
            explored[neighbor] = node
            push!(frontier, neighbor)
        end
    end

    # return the paths from the start to all nearest enemy nodes
    path(n) = haskey(explored, n) ? [path(explored[n]); n] : [n]
    return [path(n) for n in nearest]
end
          
# immediate attack target lookup
function victims(grid, units, unit)
    return sort([u for u in units 
                 if u != unit 
                    && grid[u.pos] != grid[unit.pos]
                    && distance(u.pos, unit.pos) == 1],
                by=x -> (x.hp, Tuple(x.pos)))
end
                

In [62]:
# part 1

grid = copy(data)
units = [Unit(p, grid[p], 200, 3) 
         for p in findall(x -> x in [ELF, GOBLIN], grid)]

# run each turn of the game, ignoring partial turns
done = false
partial = false
i = 0
while !done
    # run each unit, in reading order from top to bottom and left to right
    sort!(units, by=x -> Tuple(x.pos))
    j = 1
    while j <= length(units)
        unit = units[j]
        
        # locate any immediately attackable units
        targets = victims(grid, units, unit)

        # move
        if isempty(targets)
            # find the paths to the nearest enemies
            targets = search(grid, unit.pos)
            if !isempty(targets)
                # take the first step in the direction of the nearest target
                step = sort([x[2] for x in targets], by=Tuple)[1]
                grid[step] = grid[unit.pos]
                grid[unit.pos] = FLOOR
                unit.pos = step
                                
                # see if we can now attack any target
                targets = victims(grid, units, unit)
            end
        end
            
        # attack
        if !isempty(targets)
            target = targets[1]
            target.hp -= unit.ap
            if target.hp <= 0
                # remove the dead unit from the board
                k = findfirst(u -> u == target, units)
                deleteat!(units, k)
                grid[target.pos] = FLOOR
                if k < j
                    j -= 1
                end

                # if all enemy units are dead the game is over
                if length(Set(u.race for u in units)) == 1
                    done = true
                    if j < length(units)
                        partial = true
                    end
                end
            end
        end
        j += 1
    end

    # don't count partial turns
    if !partial
        i += 1
    end
end

i * sum(u.hp for u in units)


191575

In [54]:
# part 2

# run a simulation of the game for a given elf attack points
function simulate(elfap)
    grid = copy(data)
    units = [Unit(p, grid[p], 200, grid[p] == ELF ? elfap : 3)
             for p in findall(x -> x in [ELF, GOBLIN], grid)]

    # run each turn of the game, ignoring partial turns
    done = false
    partial = false
    i = 0
    while !done
        # run each unit, in reading order from top to bottom and left to right
        sort!(units, by=u -> Tuple(u.pos))
        j = 1
        while j <= length(units)
            unit = units[j]
            
            # locate any immediately attackable units
            targets = victims(grid, units, unit)

            # move
            if isempty(targets)
                # find the paths to the nearest enemies
                targets = search(grid, unit.pos)
                if !isempty(targets)
                    # take the first step in the direction of the nearest target
                    step = sort([x[2] for x in targets], by=Tuple)[1]
                    grid[step] = grid[unit.pos]
                    grid[unit.pos] = FLOOR
                    unit.pos = step

                    # see if we can now attack any target
                    targets = victims(grid, units, unit)
                end
            end

            # attack
            if !isempty(targets)
                target = targets[1]
                target.hp -= unit.ap
                if target.hp <= 0
                    # no elves can die
                    if target.race == ELF
                        return 0
                    end
                        
                    # remove the dead unit from the board
                    k = findfirst(u -> u == target, units)
                    deleteat!(units, k)
                    grid[target.pos] = FLOOR
                    if k < j
                        j -= 1
                    end

                    # if all enemy units are dead the game is over
                    if length(Set(u.race for u in units)) == 1
                        done = true
                        if j < length(units)
                            partial = true
                        end
                    end
                end
            end
            j += 1
        end

        # don't count partial turns
        if !partial
            i += 1
        end
    end
                                                
    return i * sum(u.hp for u in units)
end

# bisect to find a game with no elf deaths
start = stop = 4
while simulate(stop) == 0
    start = stop
    stop *= 2
end

# linear search to find the first game with no elf deaths
for i in start + 1:stop
    sim = simulate(i)
    if sim != 0 return sim end
end


75915